In [53]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [54]:
# Use only the derived coefficients
# For Li+
coffe1_n = pybamm.FullBroadcast(
        pybamm.Scalar(3.75e-11), "negative electrode", )
coffe1_s = pybamm.FullBroadcast(
        pybamm.Scalar(9.666472986565472e-11), "separator", )
coffe1_p = pybamm.FullBroadcast(
        pybamm.Scalar(5.816858043652089e-11), "positive electrode", )
coffe1 = pybamm.concatenation(coffe1_n, coffe1_s, coffe1_p )
coffe2_n = pybamm.FullBroadcast(
        pybamm.Scalar(-3.1249999999999995e-11), "negative electrode", )
coffe2_s = pybamm.FullBroadcast(
        pybamm.Scalar(-8.055394155471226e-11), "separator", )
coffe2_p = pybamm.FullBroadcast(
        pybamm.Scalar(-4.847381703043407e-11), "positive electrode", )
coffe2 = pybamm.concatenation(coffe2_n, coffe2_s, coffe2_p )
coffe3_n = pybamm.FullBroadcast(
        pybamm.Scalar(1.9136951216600957), "negative electrode", )
coffe3_s = pybamm.FullBroadcast(
        pybamm.Scalar(0), "separator", )
coffe3_p = pybamm.FullBroadcast(
        pybamm.Scalar(-2.3745495341343483), "positive electrode", )
coffe3 = pybamm.concatenation(coffe3_n, coffe3_s, coffe3_p )
# For EC
coffEC1_n = pybamm.FullBroadcast(
        pybamm.Scalar(-1.2499999999999998e-10), "negative electrode", )
coffEC1_s = pybamm.FullBroadcast(
        pybamm.Scalar(-3.2221576621884903e-10), "separator", )
coffEC1_p = pybamm.FullBroadcast(
        pybamm.Scalar(-1.9389526812173628e-10), "positive electrode", )
coffEC1 = pybamm.concatenation(coffEC1_n, coffEC1_s, coffEC1_p )
coffEC2_n = pybamm.FullBroadcast(
        pybamm.Scalar(6.25e-11), "negative electrode", )
coffEC2_s = pybamm.FullBroadcast(
        pybamm.Scalar(1.6110788310942454e-10), "separator", )
coffEC2_p = pybamm.FullBroadcast(
        pybamm.Scalar(9.694763406086815e-11), "positive electrode", )
coffEC2 = pybamm.concatenation(coffEC2_n, coffEC2_s, coffEC2_p )

coffEC3_n = pybamm.FullBroadcast(
        pybamm.Scalar(-4.045348278871794), "negative electrode", )
coffEC3_s = pybamm.FullBroadcast(
        pybamm.Scalar(0), "separator", )
coffEC3_p = pybamm.FullBroadcast(
        pybamm.Scalar(0), "positive electrode", )
coffEC3 = pybamm.concatenation(coffEC3_n, coffEC3_s, coffEC3_p )

In [55]:
model = pybamm.BaseModel()
c_e_n  = pybamm.Variable("Li+ concentration in negative electrode", domain="negative electrode")
c_EC_n = pybamm.Variable("EC concentration in negative electrode", domain="negative electrode")

c_e_s  = pybamm.Variable("Li+ concentration in separator", domain="separator")
c_EC_s = pybamm.Variable("EC concentration in separator", domain="separator")

c_e_p  = pybamm.Variable("Li+ concentration in positive electrode", domain="positive electrode")
c_EC_p = pybamm.Variable("EC concentration in positive electrode", domain="positive electrode")

c_EC = pybamm.concatenation(c_EC_n, c_EC_s, c_EC_p)
c_e  = pybamm.concatenation(c_e_n, c_e_s, c_e_p)
model.variables = {
    "Li+ concentration in negative electrode":c_e_n,
    "Li+ concentration in separator":c_e_s,
    "Li+ concentration in positive electrode":c_e_p,
    "EC concentration in negative electrode":c_EC_n,
    "EC concentration in separator":c_EC_s,
    "EC concentration in positive electrode":c_EC_p,
}

In [56]:
model.variables.update(
    {
        "Li+ concentration": c_e, 
        "EC concentration": c_EC,
    }
)

In [57]:
print(model.variables.keys())

dict_keys(['Li+ concentration in negative electrode', 'Li+ concentration in separator', 'Li+ concentration in positive electrode', 'EC concentration in negative electrode', 'EC concentration in separator', 'EC concentration in positive electrode', 'Li+ concentration', 'EC concentration'])


In [58]:
model.length_scales = {
    "negative electrode": 8.52e-5,
    "separator": 1.2e-5,
    "positive electrode": 7.56e-5,
}
D_e     = 3e-10;    # Li+ diffusivity in m2/s
model.timescale = pybamm.Scalar(8.52e-5 ** 2 /  D_e) 

In [59]:
model.rhs = {
    c_e: 
    coffe1  * pybamm.div(pybamm.grad(c_e))
    +coffe2 * pybamm.div(pybamm.grad(c_EC))
    + coffe3,

    c_EC:
    coffEC1   * pybamm.div(pybamm.grad(c_e))
    + coffEC2 * pybamm.div(pybamm.grad(c_EC))
    + coffEC3,
} 

In [60]:
print(coffe1)

concatenation(broadcast(3.75e-11), broadcast(9.666472986565472e-11), broadcast(5.816858043652089e-11))


In [61]:
model.initial_conditions = {c_e: pybamm.Scalar(1000),c_EC: pybamm.Scalar(4000)}

model.boundary_conditions = {
    c_e: {"left": (pybamm.Scalar(0), "Neumann"),"right": (pybamm.Scalar(0), "Neumann")},  
    c_EC:{"left": (pybamm.Scalar(0), "Neumann"),"right": (pybamm.Scalar(0), "Neumann")} }
# define geometry
x_n = pybamm.SpatialVariable(
    "x_n", 
    domain=["negative electrode"], 
    coord_sys="cartesian"
)
x_s = pybamm.SpatialVariable(
    "x_s", 
    domain=["separator"], 
    coord_sys="cartesian"
)
x_p = pybamm.SpatialVariable(
    "x_p", 
    domain=["positive electrode"], 
    coord_sys="cartesian"
)
geometry = {
    "negative electrode": {x_n: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(8.52e-5)}},
    "separator":          {x_s: {"min": pybamm.Scalar(8.52e-5), "max": pybamm.Scalar(9.72e-5)}},
    "positive electrode": {x_p: {"min": pybamm.Scalar(9.72e-5), "max": pybamm.Scalar(17.28e-5)}},
}
# mesh and discretise ,"separator","positive electrode"
submesh_types = {
    "negative electrode": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
    "separator": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
    "positive electrode": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
}
var_pts = {x_n: 20, x_s: 20, x_p: 20}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {
    "negative electrode": pybamm.FiniteVolume(),
    "separator": pybamm.FiniteVolume(),
    "positive electrode": pybamm.FiniteVolume(),
}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

In [63]:
# solve
solver = pybamm.ScipySolver()
#t = np.linspace(0, 100, 50)
solution = solver.solve(model, [0,1])


In [64]:
pybamm.dynamic_plot(
    solution,
    [
        "Li+ concentration",
        "EC concentration",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…